In [41]:
# System libraries:
%load_ext autoreload
import pandas as pd
import json
import nltk
import numpy as np
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime
import re
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
def import_integrum():
    data = pd.read_csv('data/integrum_russian_mentionning_finaland.csv')
    rename_cols = {'Title':'headline', 'Text':'content', 'ID':'id', 'Date':'date'}

    data = data.rename(rename_cols, axis = 'columns')
    
    return data
    
#data = import_integrum()

#data.to_csv('/var/www/html/work/compressed/data/russia_finland/processed/integrum_processed.csv', index = False)
#lemmas = pd.read_csv('data/processed/integrum_lemmas.csv', converters = {'lemmas_content' : eval, 'lemmas_headline':eval})
data = pd.concat([data, lemmas], axis = 1)

In [39]:
data.iloc[:5]

,id,date,MediaDetail,MediaShort,headline,content,lemmas_headline,lemmas_content
0,1,01.11.2003,Известия (Москва),Известия,"Владимир ВУЙТЕК, главный тренер хоккейного клу...",О чешском тренере Владимире ВУЙТЕКЕ нынче прин...,"[министр, окружающий, среда, ниинист, хотеть, ...","[министр, вопрос, окружающий, среда, вилла, ни..."
1,2,03.11.2003,Известия (Москва),Известия,Судьба Маргариты,В минувшее воскресенье исполнилось 110 лет со ...,"[финский, труд, дешево, труд, германия]","[труд, финский, работник, занимать, 8, дорогов..."
2,3,04.11.2003,Известия (Москва),Известия,Девушка в кокошнике - лицо России,В минувшие выходные в Греции завершилась 19-я ...,"[начинаться, постепенный, переход, летний, огр...","[на, дорога, южный, финляндия, летний, огранич..."
3,4,05.11.2003,Известия (Москва),Известия,Не прошло и полвека,Сотрудники МИД Финляндии разыскали живого свид...,"[в, надворный, суд, начинаться, разбирательств...","[в, хельсинкский, надворный, суд, начинаться, ..."
4,5,06.11.2003,Известия (Москва),Известия,ХРОНИКА,EXXON MOBIL НАМЕРЕНА РАЗВИВАТЬ СОТРУДНИЧЕСТВО ...,"[летний, рабочий, место, хватать]","[несмотря, усилие, различный, кампания, создан..."


In [33]:
mod = False
data = None
if mod:
    print("Regenerating data")
    data = import_integrum()
else:
    print("Loading from CSV")
    data = pd.read_csv('data/processed/integrum_processed.csv')
    

Loading from CSV


In [34]:
data.iloc[:5]

,id,date,MediaDetail,MediaShort,headline,content
0,1,01.11.2003,Известия (Москва),Известия,"Владимир ВУЙТЕК, главный тренер хоккейного клу...",О чешском тренере Владимире ВУЙТЕКЕ нынче прин...
1,2,03.11.2003,Известия (Москва),Известия,Судьба Маргариты,В минувшее воскресенье исполнилось 110 лет со ...
2,3,04.11.2003,Известия (Москва),Известия,Девушка в кокошнике - лицо России,В минувшие выходные в Греции завершилась 19-я ...
3,4,05.11.2003,Известия (Москва),Известия,Не прошло и полвека,Сотрудники МИД Финляндии разыскали живого свид...
4,5,06.11.2003,Известия (Москва),Известия,ХРОНИКА,EXXON MOBIL НАМЕРЕНА РАЗВИВАТЬ СОТРУДНИЧЕСТВО ...


In [47]:
# Lemmatization functions:
stop_words = stopwords.words('russian') + list(string.punctuation) + ['»','«']
m = Mystem()
integer_regex = re.compile('^[-+]?[0-9]+$')

def tokenize_text(text):
    word_tokens = word_tokenize(text)
    tokenized_sentence = [w for w in word_tokens if not w in stop_words and not integer_regex.match(w)]
    return tokenized_sentence
    
def lemmatize_text(text):
    return [m.lemmatize(w)[0] for w in text]

def tokenize_and_lemmatize(text):
    tokenized = tokenize_text(text)
    lemmatized = lemmatize_text(tokenized)
    return lemmatized

In [49]:
data['tokens_content'] = data.content.apply(tokenize_text)
data['tokens_headline'] = data.headline.apply(tokenize_text)

In [51]:
data.to_csv('data/processed/integrum_processed.csv', header = True, index = False)

In [64]:
# Filter by local and federal media
# Local media - 
# Federal media - Известия Коммерсантъ Комсомольская правда Независимая газета Российская газета

# Create 2 data frames - one with federal media and one with regional media 
# Save them to separate CSVs

federal_filter = ['Известия', 'Коммерсантъ', 'Комсомольская Правда',
       'Независимая Газета', 'Российская Газета']

local_filter = ['Деловой Петербург',
       'Санкт-Петербургские Ведомости', 'Карелия (Петрозаводск)',
       'Карельская губерния (Петрозаводск)',
       'Коммерсантъ (Санкт-Петербург)']

integrum_federal = data[data['MediaShort'].isin(federal_filter)]
integrum_local = data[data['MediaShort'].isin(local_filter)]

integrum_federal.to_csv('data/processed/integrum_federal.csv', header = True, index = False)
integrum_local.to_csv('data/processed/integrum_local.csv', header = True, index = False)

In [65]:
integrum_local.iloc[:5]

,id,date,MediaDetail,MediaShort,headline,content,lemmas_headline,lemmas_content,tokens_content,tokens_headline
25379,25380,05.06.2000,Деловой Петербург,Деловой Петербург,Таможню разблокировали,Начальник Выборгской таможни Иван Иоч дал устн...,"[скандинавский, социал-демократ, собираться, х...","[социал-демократ, скандинавский, страна, собир...","[Начальник, Выборгской, таможни, Иван, Иоч, да...","[Таможню, разблокировали]"
25380,25381,05.06.2000,Деловой Петербург,Деловой Петербург,Чем хуже-тем лучше,Только замедление темпов роста американской эк...,"[вехвиляйнен, покидать, Finnair, переходить, р...","[исполнительный, директор, национальный, авиак...","[Только, замедление, темпов, роста, американск...","[Чем, хуже-тем]"
25381,25382,09.06.2000,Деловой Петербург,Деловой Петербург,&lt;Газпром&gt; протянул ветку,ОАО &lt;Газпром&gt; завершило длившееся более ...,"[надворный, суд, рассматривать, обвинение, тяж...","[в, надворный, суд, турку, сегодня, начинаться...","[ОАО, lt, Газпром, gt, завершило, длившееся, л...","[lt, Газпром, gt, протянул, ветку]"
25382,25383,09.06.2000,Деловой Петербург,Деловой Петербург,Бизнесмены дрались клюшками,Во Дворце спорта СКА в хоккей играли на-стоящи...,"[в, финляндия, установленный, новый, рекорд, м...","[в, лапландия, установленный, новый, «\n, моро...","[Во, Дворце, спорта, СКА, хоккей, играли, на-с...","[Бизнесмены, дрались, клюшками]"
25383,25384,14.06.2000,Деловой Петербург,Деловой Петербург,Kvaerner исчез с карты области,На собрании акционеров ОАО &lt;Квернер Выборг ...,"[министерство, образование, обманывать, ожидан...","[министерство, образование, выделять, дополнит...","[На, собрании, акционеров, ОАО, lt, Квернер, В...","[Kvaerner, исчез, карты, области]"


In [75]:
integrum_federal.iloc[:100].tokens_content.apply(lemmatize_text)

0     [о, чешский, тренер, владимир, вуйтек, нынче, ...
1     [в, минувший, воскресение, исполняться, год, д...
2     [в, минувший, выходной, греция, завершаться, 1...
3     [сотрудник, мид, финляндия, разыскивать, живой...
4     [EXXON, MOBIL, намерен, развивать, сотрудничес...
5     [защитник, омский, ``\n, авангард, ''\n, олег,...
6     [сегодня, вечер, сборная, россия, сыграть, хел...
7     [авиационный, служба, россия, не, смочь, догов...
8     [зима, улица, российский, город, выливать, 1, ...
9     [одерживать, один, убедительный, победа, дважд...
10    [баскетбол, евролига, улеб, мужчина, первый, г...
11    [молодость, свойственный, уходить, возвращатьс...
12    [среди, армия, валлиец, который, конец, следую...
13    [NOKIA, раздумывать, над, покупка, PSION, по, ...
14    [известный, скульптор, зураб, церетель, готовы...
15    [не, плакать, аргентина, анонсировать, ``\n, и...
16    [сегодня, премьер, михаил, касьянов, начинать,...
17    [сегодня, переговоры, премьера, михаил, ка

In [72]:
data = None

In [77]:
# lemmatize content:
integrum_federal['lemmas_content'] = integrum_federal.tokens_content.apply(lemmatize_text)

# lemmatize headlines:
integrum_federal['lemmas_headline'] = integrum_federal.tokens_headline.apply(lemmatize_text)

/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [91]:
integrum_federal[['year','lemmas_content','lemmas_headline']].to_csv('data/processed/integrum_federal_lemmas.csv', header = True, index = False)



In [92]:
integrum_federal.to_csv('data/processed/integrum_federal.csv', header = True, index = False)

In [83]:
# lemmatize content:
integrum_local['lemmas_content'] = integrum_local.tokens_content.apply(lemmatize_text)

# lemmatize headlines:
integrum_local['lemmas_headline'] = integrum_local.tokens_headline.apply(lemmatize_text)

/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [89]:
integrum_local.iloc[:5].date

type(integrum_local.iloc[0].date)
datestring = integrum_local.iloc[0].date
datestring


year = datetime.strptime(datestring[:10], '%d.%m.%Y').year

year

integrum_local['year'] = integrum_local['date'].apply(lambda s: datetime.strptime(s[:10], '%d.%m.%Y').year)
integrum_local.iloc[:5]


25379    05.06.2000
25380    05.06.2000
25381    09.06.2000
25382    09.06.2000
25383    14.06.2000
Name: date, dtype: object

str

'05.06.2000'

2000

/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,id,date,MediaDetail,MediaShort,headline,content,lemmas_headline,lemmas_content,tokens_content,tokens_headline,year
25379,25380,05.06.2000,Деловой Петербург,Деловой Петербург,Таможню разблокировали,Начальник Выборгской таможни Иван Иоч дал устн...,"[таможня, разблокировать]","[начальник, выборгский, таможня, иван, иоч, да...","[Начальник, Выборгской, таможни, Иван, Иоч, да...","[Таможню, разблокировали]",2000
25380,25381,05.06.2000,Деловой Петербург,Деловой Петербург,Чем хуже-тем лучше,Только замедление темпов роста американской эк...,"[что, плохо]","[только, замедление, темп, рост, американский,...","[Только, замедление, темпов, роста, американск...","[Чем, хуже-тем]",2000
25381,25382,09.06.2000,Деловой Петербург,Деловой Петербург,&lt;Газпром&gt; протянул ветку,ОАО &lt;Газпром&gt; завершило длившееся более ...,"[lt, газпром, gt, протягивать, ветка]","[оао, lt, газпром, gt, завершать, длиться, год...","[ОАО, lt, Газпром, gt, завершило, длившееся, л...","[lt, Газпром, gt, протянул, ветку]",2000
25382,25383,09.06.2000,Деловой Петербург,Деловой Петербург,Бизнесмены дрались клюшками,Во Дворце спорта СКА в хоккей играли на-стоящи...,"[бизнесмен, драться, клюшка]","[во, дворец, спорт, ска, хоккей, играть, на, м...","[Во, Дворце, спорта, СКА, хоккей, играли, на-с...","[Бизнесмены, дрались, клюшками]",2000
25383,25384,14.06.2000,Деловой Петербург,Деловой Петербург,Kvaerner исчез с карты области,На собрании акционеров ОАО &lt;Квернер Выборг ...,"[Kvaerner, исчезать, карта, область]","[на, собрание, акционер, оао, lt, квернер, выб...","[На, собрании, акционеров, ОАО, lt, Квернер, В...","[Kvaerner, исчез, карты, области]",2000


In [90]:
integrum_federal['year'] = integrum_federal['date'].apply(lambda s: datetime.strptime(s[:10], '%d.%m.%Y').year)
integrum_federal.iloc[:5]

/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,date,MediaDetail,MediaShort,headline,content,lemmas_headline,lemmas_content,tokens_content,tokens_headline,year
0,1,01.11.2003,Известия (Москва),Известия,"Владимир ВУЙТЕК, главный тренер хоккейного клу...",О чешском тренере Владимире ВУЙТЕКЕ нынче прин...,"[владимир, вуйтек, главный, тренер, хоккейный,...","[о, чешский, тренер, владимир, вуйтек, нынче, ...","[О, чешском, тренере, Владимире, ВУЙТЕКЕ, нынч...","[Владимир, ВУЙТЕК, главный, тренер, хоккейного...",2003
1,2,03.11.2003,Известия (Москва),Известия,Судьба Маргариты,В минувшее воскресенье исполнилось 110 лет со ...,"[судьба, маргарита]","[в, минувший, воскресение, исполняться, год, д...","[В, минувшее, воскресенье, исполнилось, лет, д...","[Судьба, Маргариты]",2003
2,3,04.11.2003,Известия (Москва),Известия,Девушка в кокошнике - лицо России,В минувшие выходные в Греции завершилась 19-я ...,"[девушка, кокошник, лицо, россия]","[в, минувший, выходной, греция, завершаться, 1...","[В, минувшие, выходные, Греции, завершилась, 1...","[Девушка, кокошнике, лицо, России]",2003
3,4,05.11.2003,Известия (Москва),Известия,Не прошло и полвека,Сотрудники МИД Финляндии разыскали живого свид...,"[не, проходить, полвека]","[сотрудник, мид, финляндия, разыскивать, живой...","[Сотрудники, МИД, Финляндии, разыскали, живого...","[Не, прошло, полвека]",2003
4,5,06.11.2003,Известия (Москва),Известия,ХРОНИКА,EXXON MOBIL НАМЕРЕНА РАЗВИВАТЬ СОТРУДНИЧЕСТВО ...,[хроника],"[EXXON, MOBIL, намерен, развивать, сотрудничес...","[EXXON, MOBIL, НАМЕРЕНА, РАЗВИВАТЬ, СОТРУДНИЧЕ...",[ХРОНИКА],2003


In [93]:
integrum_local.to_csv('data/processed/integrum_local.csv', header = True, index = False)

In [94]:
integrum_local[['year','lemmas_content','lemmas_headline']].to_csv('data/processed/integrum_local_lemmas.csv', header = True, index = False)

In [120]:
integrum_federal.iloc[:5]

,id,date,MediaDetail,MediaShort,headline,content,lemmas_headline,lemmas_content,tokens_content,tokens_headline,year
0,1,01.11.2003,Известия (Москва),Известия,"Владимир ВУЙТЕК, главный тренер хоккейного клу...",О чешском тренере Владимире ВУЙТЕКЕ нынче прин...,"[владимир, вуйтек, главный, тренер, хоккейный,...","[о, чешский, тренер, владимир, вуйтек, нынче, ...","[О, чешском, тренере, Владимире, ВУЙТЕКЕ, нынч...","[Владимир, ВУЙТЕК, главный, тренер, хоккейного...",2003
1,2,03.11.2003,Известия (Москва),Известия,Судьба Маргариты,В минувшее воскресенье исполнилось 110 лет со ...,"[судьба, маргарита]","[в, минувший, воскресение, исполняться, год, д...","[В, минувшее, воскресенье, исполнилось, лет, д...","[Судьба, Маргариты]",2003
2,3,04.11.2003,Известия (Москва),Известия,Девушка в кокошнике - лицо России,В минувшие выходные в Греции завершилась 19-я ...,"[девушка, кокошник, лицо, россия]","[в, минувший, выходной, греция, завершаться, 1...","[В, минувшие, выходные, Греции, завершилась, 1...","[Девушка, кокошнике, лицо, России]",2003
3,4,05.11.2003,Известия (Москва),Известия,Не прошло и полвека,Сотрудники МИД Финляндии разыскали живого свид...,"[не, проходить, полвека]","[сотрудник, мид, финляндия, разыскивать, живой...","[Сотрудники, МИД, Финляндии, разыскали, живого...","[Не, прошло, полвека]",2003
4,5,06.11.2003,Известия (Москва),Известия,ХРОНИКА,EXXON MOBIL НАМЕРЕНА РАЗВИВАТЬ СОТРУДНИЧЕСТВО ...,[хроника],"[EXXON, MOBIL, намерен, развивать, сотрудничес...","[EXXON, MOBIL, НАМЕРЕНА, РАЗВИВАТЬ, СОТРУДНИЧЕ...",[ХРОНИКА],2003


In [127]:
# Target 1 - get a word cloud using TF-IDF for the whole dataset
N = 1000
corpus = integrum_federal.lemmas_content.apply(lambda s: ' '.join(s))

def tf_idf_score(corpus):
    tf = TfidfVectorizer(min_df = 0, stop_words = None)
    
    tfidf_matrix = tf.fit_transform(corpus)
    feature_names = tf.get_feature_names() 
    
    top_words = pd.Series(index=range(0,len(corpus)))
    
    for i, row in enumerate(tfidf_matrix):
        idx = np.argsort(np.array(row.todense()))[:,-20:]
        words = np.array(feature_names)[idx]
        top_words.iloc[i] = words[0]
    return top_words

integrum_federal_tf_idf = tf_idf_score(corpus)
#data['tfidf_top_words'] = tf_idf_words

KeyboardInterrupt: 

In [96]:
corpus

0     о чешский тренер владимир вуйтек нынче принима...
1     в минувший воскресение исполняться год день ро...
2     в минувший выходной греция завершаться 19 межд...
3     сотрудник мид финляндия разыскивать живой свид...
4     EXXON MOBIL намерен развивать сотрудничество с...
5     защитник омский ``\n авангард ''\n олег твердо...
6     сегодня вечер сборная россия сыграть хельсинки...
7     авиационный служба россия не смочь договариват...
8     зима улица российский город выливать 1 миллион...
9     одерживать один убедительный победа дважды неу...
10    баскетбол евролига улеб мужчина первый группов...
11    молодость свойственный уходить возвращаться пр...
12    среди армия валлиец который конец следующий не...
13    NOKIA раздумывать над покупка PSION по сообщен...
14    известный скульптор зураб церетель готовый реа...
15    не плакать аргентина анонсировать ``\n извести...
16    сегодня премьер михаил касьянов начинать визит...
17    сегодня переговоры премьера михаил касьяно

In [ ]:
data.iloc[:10]['tfidf_top_words']

In [ ]:
# Filter by local and federal media
# Local media - 
# Federal media - Известия Коммерсантъ Комсомольская правда Независимая газета Российская газета

# Create 2 data frames - one with federal media and one with regional media 
# Save them to separate CSVs

In [ ]:
# Get the year
type(data.iloc[0].datePublished)
datestring = data.iloc[0].datePublished
datestring


year = datetime.strptime(datestring[:10], '%Y-%m-%d').year

year

data['year'] = data['datePublished'].apply(lambda s: datetime.strptime(s[:10], '%Y-%m-%d').year)
data.iloc[:5]

In [ ]:
# Target 1 - continued - get a word cloud using TF-IDF for the whole dataset

# Concatenate selected lists in the data['tfidf_top_words'] and generate a histogram

def get_tf_idf_counts(series):
    tf_idf_flattened = [b for a in series for b in a]
    counts = Counter(tf_idf_flattened)
    return counts

def output_wordcloud(counts, title = None):
    cloud = WordCloud(stopwords=None, background_color='black', width=2500, height=1800
                     ).generate_from_frequencies(counts)

    plt.figure(figsize=(20,10))
    if title:
        plt.title(title)
    plt.imshow(cloud)
    plt.axis('off')
    plt.show()
    


In [ ]:
series = data.iloc[:100]['tfidf_top_words']
counts = get_tf_idf_counts(series)  
output_wordcloud(counts)

In [30]:
# Enter your search string:
search_string = 'Эспоо'
results = data[data['headline'].str.contains(search_string)]

# Get all results matching search string
results

,id,date,MediaDetail,MediaShort,headline,content
47136,47137,12.01.1999,санкт-петербургские ведомости,Санкт-Петербургские Ведомости,Гатчина _ первый побратим Эспоо,Гатчина _ первый побратим Эспоо Город Эспоо _ ...
49478,49479,01.09.2001,санкт-петербургские ведомости,Санкт-Петербургские Ведомости,Зверское убийство в Эспоо,Зверское убийство в Эспоо Полиция задержала по...
